In [2]:
import gist ### https://github.com/tuttieee/lear-gist-python

from PIL import Image
import imageio
from IPython.display import display, clear_output

import sys
sys.path.append("../Library/")
import image_download as imd
import image_manipulation as ima

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
load_dotenv("../.env")
import csv

import numpy as np
import db_connection as dbcon

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, roc_auc_score


#%matplotlib inline
%load_ext autoreload
%autoreload 2

### Parameters

In [3]:
# parameters
MFP_FOLDER = os.getenv('MFP_FOLDER')
EX_FOLDER = MFP_FOLDER + 'earthexplorer/'

### Load all images 

In [7]:
filenames = os.listdir(EX_FOLDER + "processedIMG/")
filenames = [filename for filename in filenames if filename.startswith("m")]
filenames[:10]

['m_4209830_se_14_1_20140907_x5632_y3584.png',
 'm_4709143_nw_15_1_20171010_x1024_y2560.png',
 'm_4210155_sw_14_1_20120710_x5120_y4608.png',
 'm_3809356_nw_15_1_20140703_x5632_y5120.png',
 'm_4210155_sw_14_1_20120710_x0_y4096.png',
 'm_4709143_nw_15_1_20171010_x5120_y3072.png',
 'm_4709143_nw_15_1_20171010_x6144_y1024.png',
 'm_4209830_se_14_1_20140907_x1024_y512.png',
 'm_4309736_nw_14_1_20140813_x5632_y3072.png',
 'm_4309736_nw_14_1_20140813_x6656_y1024.png']

In [8]:
images = []
for fname in filenames:
    imarray = np.asarray(Image.open(EX_FOLDER + "processedIMG/"+ fname))[:,:,:3]
    images.append(imarray)

In [14]:
df_images = pd.DataFrame({
    'filename': filenames,
    'image': images
})

In [15]:
df_images.head()

,filename,image
0,m_4209830_se_14_1_20140907_x5632_y3584.png,"[[[96, 112, 81], [103, 109, 83], [105, 111, 83..."
1,m_4709143_nw_15_1_20171010_x1024_y2560.png,"[[[120, 135, 88], [132, 150, 95], [129, 143, 9..."
2,m_4210155_sw_14_1_20120710_x5120_y4608.png,"[[[150, 147, 122], [152, 149, 123], [149, 149,..."
3,m_3809356_nw_15_1_20140703_x5632_y5120.png,"[[[101, 132, 74], [78, 93, 79], [116, 130, 79]..."
4,m_4210155_sw_14_1_20120710_x0_y4096.png,"[[[130, 120, 106], [132, 117, 100], [125, 114,..."


In [17]:
# Generate feature vector with gist
feature_vector = []
for image in df_images["image"].tolist():
    feature_vector.append(gist.extract(image, nblocks=1, orientations_per_scale=(8, 8, 4)))
    
feature_matrix = np.array(feature_vector)
feature_matrix.shape

(1938, 60)

In [20]:
feature_matrix_list = [feature_matrix[row,:] for row in range(feature_matrix.shape[0])]

In [21]:
df_images['gist'] = feature_matrix_list

In [23]:
df_images.to_csv(EX_FOLDER + "gist.csv", index = False)